In this notebook we will study the energy conservation properties of several high-order methods for the Hénon–Heiles system. We will se how the energy error behaves at very thight tolerances and how different techniques such as using symplectic solvers or manifold projections benchmark against each other.
The Hamiltonian for this system is given by:
$$
\mathcal{H}=\frac{1}{2}(p_1^2 + p_2^2) + \frac{1}{2}\left(q_1^2 + q_2^2 + 2q_1^2 q_2 - \frac{2}{3}q_2^3\right)
$$

In [1]:
using DifferentialEquations, Plots
using TaylorIntegration
pyplot()

T(p) = 1//2 * sum(p.^2)
V(q) = 1//2 * (q[1]^2 + q[2]^2 + 2q[1]^2 * q[2]- 2//3 * q[2]^3)
H(q, p) = T(p) + V(q)

function q̇(t, q, p, dq)
    dq[1] = p[1]
    dq[2] = p[2]
end

function ṗ(t, q, p, dp)
    dp[1] = -q[1] * (1 + 2q[2])
    dp[2] = -q[2] - (q[1]^2 - q[2]^2)
end

const q0 = [0.1, 0.]
const p0 = [0., 0.5]
const E = H(q0, p0)

function g(u, resid)
    resid[1,1] = H(u[1,:], u[2,:]) - E
    resid[1,2] = 0
    resid[2,:] .= 0
end

const cb = ManifoldProjection(g, nlopts=Dict(:ftol=>1e-13));

For the comparison we will use the following function

In [2]:
energy_err(t,u1,u2,u3,u4) = (t, H([u1,u2], [u3,u4]) - E)
energy_err(sol) = [abs.(H([sol[1,j], sol[2,j]], [sol[3,j], sol[4,j]]) - E) for j=1:size(sol,2)]

function compare(; tmax=1e2)
    prob = DynamicalODEProblem(q̇, ṗ, q0, p0, (0., tmax))
    @time sol1 = solve(prob, Vern9(), callback=cb, abstol=1e-14, reltol=1e-14)
    @time sol2 = solve(prob, KahanLi8(), dt=1e-2)
    @time sol3 = solve(prob, Vern9(), abstol=1e-14, reltol=1e-14)
    @time sol4 = solve(prob, DPRKN12(), abstol=1e-14, reltol=1e-14)
    @time sol5 = solve(prob, TaylorMethod(50), abstol=1e-20)
    
    println("Vern9 + ManifoldProjection max energy error: ", maximum(energy_err(sol1)))
    println("KahanLi8 max energy error: ",                   maximum(energy_err(sol2)))
    println("Vern9 max energy error: ",                      maximum(energy_err(sol3)))
    println("DPRKN12 max energy error: ",                    maximum(energy_err(sol4)))
    println("TaylorMethod max energy error: ",               maximum(energy_err(sol5)))
    
    plot(xlabel="t", ylabel="Energy error")
    plot!(sol1, vars=(energy_err, 0,1,2,3,4), label="Vern9 + ManifoldProjection")
    plot!(sol2, vars=(energy_err, 0,1,2,3,4), label="KahanLi8")
    plot!(sol3, vars=(energy_err, 0,1,2,3,4), label="Vern9")
    plot!(sol4, vars=(energy_err, 0,1,2,3,4), label="DPRKN12")
    plot!(sol5, vars=(energy_err, 0,1,2,3,4), label="TaylorMethod")
end

compare (generic function with 1 method)

Note: The `NLsolve` call used by `ManifoldProjection` was modified to use `ftol=1e-13` in order to obtain a very low energy error. 

In [4]:
compare(tmax=1e2)

 54.586408 seconds (300.84 M allocations: 9.440 GiB, 3.26% gc time)
  0.020354 seconds (190.20 k allocations: 10.318 MiB, 44.72% gc time)
  0.036949 seconds (1.58 M allocations: 26.838 MiB, 33.37% gc time)
  0.000726 seconds (9.00 k allocations: 407.922 KiB)


LoadError: [91mMethodError: no method matching vec(::Tuple{Array{Float64,1},Array{Float64,1}})[0m
Closest candidates are:
  vec([91m::SparseMatrixCSC[39m) at sparse/sparsematrix.jl:384
  vec([91m::AbstractSparseArray{Tv,Ti,1} where Ti where Tv[39m) at sparse/sparsevector.jl:824
  vec([91m::StaticArrays.StaticArray[39m) at /home/user/.julia/v0.6/StaticArrays/src/abstractarray.jl:122
  ...[39m

Because the energy error for the `Vern9` method is lower than the tolerance for `NLsolve`, this method and the `ManifoldProjection` version produce identical results.

In [ ]:
compare(tmax=1e3)

In [ ]:
compare(tmax=1e4)

In [ ]:
compare(tmax=5e4)

We can see that as the simulation time increases, the energy error increases. For this particular example the energy error for all the methods is comparable, but for smaller simulation times, if a highly accurate solution is required, the symplectic method is not recommended as its energy error fluctuations are relatively larger.
The above plot shows how the `Vern9` method and the `ManifoldProjection` version start to differ exactly when the energy error reaches the chosen tolerance for `NLsolve` (`ftol=1e-13`).